In [7]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')
%matplotlib inline
from tqdm import tqdm
from multitest import MultiTest
from datasets import load_dataset

In [5]:
from scipy.interpolate import interp1d
import pandas as pd

In [8]:
import numpy as np
import torch
import os
dataset = load_dataset("aadityaubhat/GPT-wiki-intro")

Using custom data configuration aadityaubhat--GPT-wiki-intro-10ad8b711a5f3880
Found cached dataset csv (/Users/kipnisal/.cache/huggingface/datasets/aadityaubhat___csv/aadityaubhat--GPT-wiki-intro-10ad8b711a5f3880/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [41]:
path = 'Data/GPT3/wiki/'
for d in tqdm(dataset['train']):
    filename = path + 'wiki_' + str(d['id']) + '.txt'
    with open(filename, "wt") as f:
        f.write(d['prompt'] + d['generated_text'])

100%|██████████| 150000/150000 [00:50<00:00, 2978.43it/s]


In [40]:
path + 'wiki_' + str(d['id']) + '.txt'

'Data/GPT3/wiki/wiki_32345102.txt'

12